In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np
import boto3
import time

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from helpers import *

In [3]:
def test_greenspace_score_function():
    
    df1 = pd.read_csv(ROOT + '/Spikes/Dash/data/final_df_no_greenspace.csv', index_col = 0)
    df2 = pd.read_csv(ROOT + '/Spikes/Dash/data/final_df_no_greenspace.csv', index_col = 0)
    df2 = df2.rename(columns={'Latitude':'latitude', 'Longitude':'longitude'})
    
    df_merged = pd.merge(df1, df2, how="left", left_on=['latitude', 'longitude'], right_on=['latitude', 'longitude'])

    start = time.time()
    df = apply_greenspace_score_function(df_merged)
    end = time.time()
    time_taken3 = round(end - start, 2)
    print('apply_greenspace_score_function complete')
    print('Time taken:', time_taken3)

    total_time_taken = time_taken1 + time_taken2 + time_taken3
    print('TOTAL time taken:', total_time_taken)
    
    df.to_csv(ROOT + '/Spikes/Dash/data/test_final_df_with_greenspace_score.csv')

    return df

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)